In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!git clone https://github.com/ItWasAllYellow/public_cs224n_gpt.git

%cd /content/public_cs224n_gpt

Cloning into 'public_cs224n_gpt'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 69 (delta 21), reused 13 (delta 13), pack-reused 32 (from 1)
Receiving objects: 100% (69/69), 30.87 MiB | 29.60 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/public_cs224n_gpt


In [ ]:
"""
Fine-tune GPT-2 to generate explanations (all options in prompt)
input,target,grade,split   →  Cross-entropy LM loss
"""

import argparse, os, torch, pandas as pd, numpy as np
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model       # custom backbone

TQDM_DISABLE=False

def seed_everything(seed=42):
    import random, numpy as np, torch, os
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

# ─────────────────────────────────────────────────────────────────── #
class GPT2Explainer(nn.Module):
    def __init__(self,args):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained(
            model=args.model_size, d=args.d, l=args.l, num_heads=args.num_heads
        )
        vocab = GPT2Tokenizer.from_pretrained(args.model_size).vocab_size
        self.lm_head = nn.Linear(args.d, vocab, bias=False)
        for p in self.parameters(): p.requires_grad=True
    def forward(self,ids,mask,labels=None):
        out = self.backbone(ids, attention_mask=mask)["last_hidden_state"]
        logits = self.lm_head(out)
        if labels is None: return logits
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )
        return loss, logits
# ─────────────────────────────────────────────────────────────────── #
class ExplDataset(Dataset):
    def __init__(self,df,tok,max_len=512):
        self.df=df.reset_index(drop=True); self.tok=tok; self.max_len=max_len
    def __len__(self): return len(self.df)
    def __getitem__(self,idx):
        r=self.df.iloc[idx]
        return r["input"], r["target"]
    def collate_fn(self,batch):
        inputs, targets = zip(*batch)
        enc_in  = self.tok(list(inputs),  padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        enc_out = self.tok(list(targets), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        labels = enc_out.input_ids.clone()
        labels[enc_out.attention_mask==0] = -100    # ignore pad
        return {"input_ids":enc_in.input_ids,
                "attention_mask":enc_in.attention_mask,
                "labels":labels}
# ─────────────────────────────────────────────────────────────────── #
def evaluate(loader,model,dev):
    model.eval(); tot_loss=0; tok=0
    with torch.no_grad():
        for bt in loader:
            ids = bt["input_ids"].to(dev)
            mask= bt["attention_mask"].to(dev)
            lbl = bt["labels"].to(dev)
            loss,_ = model(ids,mask,lbl)
            num = (lbl!=-100).sum().item()
            tot_loss += loss.item()*num; tok += num
    ppl = np.exp(tot_loss/tok)
    return ppl
# ─────────────────────────────────────────────────────────────────── #
def train(args):
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    seed_everything(args.seed)
    dev = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")

    df = pd.read_csv(args.data_path, encoding="utf-8-sig")
    train_df = df[df.split=="train"]; valid_df = df[df.split=="valid"]

    tok = GPT2Tokenizer.from_pretrained(args.model_size); tok.pad_token = tok.eos_token
    tr_ds = ExplDataset(train_df, tok, args.max_length)
    va_ds = ExplDataset(valid_df, tok, args.max_length)
    tr_ld = DataLoader(tr_ds,batch_size=args.batch_size,shuffle=True, collate_fn=tr_ds.collate_fn)
    va_ld = DataLoader(va_ds,batch_size=args.batch_size,shuffle=False, collate_fn=va_ds.collate_fn)

    model = GPT2Explainer(args).to(dev)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr)

    best = 1e9
    for ep in range(1,args.epochs+1):
        model.train(); ep_loss=0
        for bt in tqdm(tr_ld,disable=TQDM_DISABLE,desc=f"Train{ep}"):
            opt.zero_grad()
            loss,_ = model(bt["input_ids"].to(dev),
                           bt["attention_mask"].to(dev),
                           bt["labels"].to(dev))
            loss.backward(); opt.step(); ep_loss+=loss.item()
        print(f"Epoch{ep} train_loss={ep_loss/len(tr_ld):.4f}")
        ppl = evaluate(va_ld, model, dev)
        print(f"  Valid PPL={ppl:.3f}")
        if ppl<best: best=ppl; save(model,opt,args)
def save(m,o,args):
    os.makedirs(os.path.dirname(args.save_path),exist_ok=True)
    torch.save({"model":m.state_dict(),"optim":o.state_dict(),"args":args},args.save_path)
    print("Saved to",args.save_path)
def get_args():
    p=argparse.ArgumentParser()
    p.add_argument("--data_path", type=str, default="/content/drive/MyDrive/CSEG321/dataset/explanation_all_options.csv")
    p.add_argument("--model_size",type=str,default="gpt2",choices=["gpt2","gpt2-medium","gpt2-large"])
    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--max_length", type=int, default=512)
    p.add_argument("--lr",        type=float, default=5e-5)
    p.add_argument("--epochs",    type=int, default=20)
    p.add_argument("--seed",      type=int, default=42)
    p.add_argument("--use_gpu",   action="store_true")
    args=p.parse_args([])
    if args.model_size=="gpt2": args.d,args.l,args.num_heads=768,12,12
    elif args.model_size=="gpt2-medium": args.d,args.l,args.num_heads=1024,24,16
    else: args.d,args.l,args.num_heads=1280,36,20
    args.save_path="/content/drive/MyDrive/CSEG321/models/expl_all_default_20.pt"
    return args
if __name__=="__main__":
    seed_everything()
    train(get_args())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Epoch18 train_loss=5.6155
  Valid PPL=2192.596


Train19: 100%|██████████| 36/36 [02:49<00:00,  4.71s/it]


Epoch19 train_loss=5.4830
  Valid PPL=2213.454


Train20: 100%|██████████| 36/36 [02:49<00:00,  4.70s/it]


Epoch20 train_loss=5.3301
  Valid PPL=2317.271


In [ ]:
"""
Fine-tune GPT-2 to generate explanations (all options in prompt)
input,target,grade,split   →  Cross-entropy LM loss
"""

import argparse, os, torch, pandas as pd, numpy as np
import torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer
from models.gpt2 import GPT2Model       # custom backbone

TQDM_DISABLE=False

def seed_everything(seed=42):
    import random, numpy as np, torch, os
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True; torch.backends.cudnn.benchmark=False

# ─────────────────────────────────────────────────────────────────── #
class GPT2Explainer(nn.Module):
    def __init__(self,args):
        super().__init__()
        self.backbone = GPT2Model.from_pretrained(
            model=args.model_size, d=args.d, l=args.l, num_heads=args.num_heads
        )
        vocab = GPT2Tokenizer.from_pretrained(args.model_size).vocab_size
        self.lm_head = nn.Linear(args.d, vocab, bias=False)
        for p in self.parameters(): p.requires_grad=True
    def forward(self,ids,mask,labels=None):
        out = self.backbone(ids, attention_mask=mask)["last_hidden_state"]
        logits = self.lm_head(out)
        if labels is None: return logits
        loss = F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            labels.view(-1),
            ignore_index=-100
        )
        return loss, logits
# ─────────────────────────────────────────────────────────────────── #
class ExplDataset(Dataset):
    def __init__(self,df,tok,max_len=512):
        self.df=df.reset_index(drop=True); self.tok=tok; self.max_len=max_len
    def __len__(self): return len(self.df)
    def __getitem__(self,idx):
        r=self.df.iloc[idx]
        return r["input"], r["target"]
    def collate_fn(self,batch):
        inputs, targets = zip(*batch)
        enc_in  = self.tok(list(inputs),  padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        enc_out = self.tok(list(targets), padding="max_length", truncation=True,
                           max_length=self.max_len, return_tensors="pt")
        labels = enc_out.input_ids.clone()
        labels[enc_out.attention_mask==0] = -100    # ignore pad
        return {"input_ids":enc_in.input_ids,
                "attention_mask":enc_in.attention_mask,
                "labels":labels}
# ─────────────────────────────────────────────────────────────────── #
def evaluate(loader,model,dev):
    model.eval(); tot_loss=0; tok=0
    with torch.no_grad():
        for bt in loader:
            ids = bt["input_ids"].to(dev)
            mask= bt["attention_mask"].to(dev)
            lbl = bt["labels"].to(dev)
            loss,_ = model(ids,mask,lbl)
            num = (lbl!=-100).sum().item()
            tot_loss += loss.item()*num; tok += num
    ppl = np.exp(tot_loss/tok)
    return ppl
# ─────────────────────────────────────────────────────────────────── #
def train(args):
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    seed_everything(args.seed)
    dev = torch.device("cuda" if args.use_gpu and torch.cuda.is_available() else "cpu")

    df = pd.read_csv(args.data_path, encoding="utf-8-sig")
    train_df = df[df.split=="train"]; valid_df = df[df.split=="valid"]

    tok = GPT2Tokenizer.from_pretrained(args.model_size); tok.pad_token = tok.eos_token
    tr_ds = ExplDataset(train_df, tok, args.max_length)
    va_ds = ExplDataset(valid_df, tok, args.max_length)
    tr_ld = DataLoader(tr_ds,batch_size=args.batch_size,shuffle=True, collate_fn=tr_ds.collate_fn)
    va_ld = DataLoader(va_ds,batch_size=args.batch_size,shuffle=False, collate_fn=va_ds.collate_fn)

    model = GPT2Explainer(args).to(dev)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr)

    best = 1e9
    for ep in range(1,args.epochs+1):
        model.train(); ep_loss=0
        for bt in tqdm(tr_ld,disable=TQDM_DISABLE,desc=f"Train{ep}"):
            opt.zero_grad()
            loss,_ = model(bt["input_ids"].to(dev),
                           bt["attention_mask"].to(dev),
                           bt["labels"].to(dev))
            loss.backward(); opt.step(); ep_loss+=loss.item()
        print(f"Epoch{ep} train_loss={ep_loss/len(tr_ld):.4f}")
        ppl = evaluate(va_ld, model, dev)
        print(f"  Valid PPL={ppl:.3f}")
        if ppl<best: best=ppl; save(model,opt,args)
def save(m,o,args):
    os.makedirs(os.path.dirname(args.save_path),exist_ok=True)
    torch.save({"model":m.state_dict(),"optim":o.state_dict(),"args":args},args.save_path)
    print("Saved to",args.save_path)
def get_args():
    p=argparse.ArgumentParser()
    p.add_argument("--data_path", type=str, default="/content/drive/MyDrive/CSEG321/dataset/explanation_only_answer.csv")
    p.add_argument("--model_size",type=str,default="gpt2",choices=["gpt2","gpt2-medium","gpt2-large"])
    p.add_argument("--batch_size", type=int, default=4)
    p.add_argument("--max_length", type=int, default=512)
    p.add_argument("--lr",        type=float, default=5e-5)
    p.add_argument("--epochs",    type=int, default=20)
    p.add_argument("--seed",      type=int, default=42)
    p.add_argument("--use_gpu",   action="store_true")
    args=p.parse_args([])
    if args.model_size=="gpt2": args.d,args.l,args.num_heads=768,12,12
    elif args.model_size=="gpt2-medium": args.d,args.l,args.num_heads=1024,24,16
    else: args.d,args.l,args.num_heads=1280,36,20
    args.save_path="/content/drive/MyDrive/CSEG321/models/expl_only_answer_default_20.pt"
    return args
if __name__=="__main__":
    seed_everything()
    train(get_args())


Train1: 100%|██████████| 36/36 [02:47<00:00,  4.65s/it]


Epoch1 train_loss=11.4924
  Valid PPL=30103.460
Saved to /content/drive/MyDrive/CSEG321//expl_only_answer_default_20.pt


Train2: 100%|██████████| 36/36 [02:50<00:00,  4.74s/it]


Epoch2 train_loss=8.8267
  Valid PPL=2389.689
Saved to /content/drive/MyDrive/CSEG321//expl_only_answer_default_20.pt


Train3: 100%|██████████| 36/36 [03:06<00:00,  5.18s/it]


...

Epoch18 train_loss=5.7071
  Valid PPL=2158.972


Train19: 100%|██████████| 36/36 [02:49<00:00,  4.71s/it]


Epoch19 train_loss=5.5759
  Valid PPL=2248.253


Train20: 100%|██████████| 36/36 [02:49<00:00,  4.71s/it]


Epoch20 train_loss=5.4213
  Valid PPL=2301.450
